# 모니터링과 디버깅

## 모니터링 범위
- 스파크 <strong>잡의 어느 지점에서 오류</strong>가 발생했는지 파악하려면 <strong>스파크 잡을 모니터링</strong>해야함

### 모니터링 대상 컴포넌트
#### 스파크 애플리케이션과 잡
- 클러스터에서 사용자 애플리케이션이 실행되는 <strong>상황을 파악하거나 디버깅하려면 가장 먼저 스파크 모니터링 도구를 사용해야함</strong>
- 스파크 모니터링 도구
  - <strong>스파크 UI</strong>
  - <strong>스파크 로그</strong>
- 스파크 UI와 스파크 로그는 실행 중인 애플리케이션의 <strong>RDD와 쿼리 실행 계획 같은 개념적 수준의 정보를 제공함</strong>

<br>

#### JVM
- 스파크는 <strong>모든 익스큐터를 개별 자바 가상 머신(JVM)에서 실행</strong>함
- <strong>저수준 디버깅</strong>이 필요하다면 JVM도구가 스파크 모니터링 도구보다 유용
- JVM 도구
  - jstack: 스택 트레이스 제공
  - jmap: 힙 덤프 생성
  - jstat: 시계열 통계 리포트 제공
  - jconsole: JVM 속성변수를 시각화된 방식으로 탐색 가능하게 함
  - etc.

<br>

#### OS와 머신
- <strong>JVM은 호스트 OS에서 실행되므로 머신의 상태를 모니터링</strong>해서 정상 작동 중인지 확인하는 것은 매우 중요
- <strong>자원(CPU, 네트웤, I/O 등)</strong>에 대한 모니터링도 함께 해야함
- 이러한 모니터링 요소들은 <strong>클러스터 수준 모니터링 솔루션</strong>에서 확인 가능
  - dstat, iostat, iotop 같은 명령을 사용하면 세밀한 모니터링 가능

<br>

#### 클러스터
- <strong>스파크 애플리케이션이 실행되는 클러스터도 모니터링</strong>해야함
- <strong>모니터링 대상: YARN, 메소스, 스탠드얼론 클러스터 매니저</strong>
- 클러스터 모니터링 솔루션을 활용하면 클러스터가 동작하지 않는 상황을 빠르게 알 수 있음
  - Ganglia, Prometheus 등

## 모니터링 대상
- 모니터링 대상은 크게 두 가지로 나눌 수 있음
  - 실행 중인 사용자 애플리케이션의 <strong>프로세스</strong>(CPU, 메모리 사용률 등)
  - <strong>프로세스 내부</strong>에서의 쿼리 실행 과정(ex) 잡과 태스크)

### 드라이버와 익스큐터 프로세스
- 스파크 애플리케이션을 모니터링할 땐 <strong>드라이버</strong>를 유심히 관찰해야함
 - 드라이버에는 모든 애플리케이션의 <strong>상태가 보관</strong>되어 있으며 안정적으로 실행 중인지 확인 가능
- <strong>익스큐터</strong>의 상태를 파악하는 것도 매우 중요
  - 스파크는 수월한 모니터링을 지원하기 위해 <strong>드롭위자드 메트릭 라이브러리 기반의 메트릭 시스템</strong>을 갖추고 있음
    - [드롭위자드](https://www.dropwizard.io/en/latest/)

### 쿼리, 잡, 스테이지, 태스크
- 특정 쿼리에서 무슨 일이 일어나기 위해 다음 각각의 정보를 알아야함
  - 쿼리
  - 잡
  - 스테이지
  - 테스크
  
<img src="https://rtfmplz.github.io/images/posts/what-is-job-stage-task-in-spark/spark-rdd-partitions-job-stage-tasks.png"/>

## 스파크 로그
- 스파크를 <strong>가장 상세하게 모니터링</strong>하는 방법 중 하나는 로그 파일을 살펴보는 것
- 스파크 애플리케이션의 로그나 스파크 자체의 로그에서 발견된 이상한 이벤트는 <strong>잡의 실패 지점이나 원인 파악</strong>을 도움
- 클러스터에서 스파크를 실행한다면 클러스터 매니저로 파일에 로그를 저장할 수 있음
-

## 스파크 UI
- 스파크 UI는 실행 중인 애플리케이션과 스파크 워크로드에 대한 <strong>평가지표를 모니터링</strong>할 수 있는 화면 제공
- 스파크 UI 탭
  - Jobs: 스파크 잡에 대한 정보 제공
  - Stages: 개별 스테이지(스테이지의 태스크를 포함)와 관련된 정보 제공
  - Storage: 스파크 애플리케이션에 캐싱된 정보와 데이터 정보 제공
  - Environment: 스파크 애플리케이션의 구성과 설정 관련 정보 제공
  - Executors: 애플리케이션에서 사용 중인 익스큐터의 상세 정보 제공
  - SQL: SQL과 DataFrame을 포함한 구조적 API 쿼리 정보 제공